#### Friday, November 8, 2024

[How to use chat models to call tools](https://python.langchain.com/docs/how_to/tool_calling/) plus [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

mamba activate langchain

This runs without errors but with unexpected results.

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo") # "gpt-4o-mini")

llm = ChatOpenAI(base_url="http://localhost:1234/v1", 
                   model = "hermes-3-llama-3.1-8b",  # do not pass in an unrecognized model name ... 
                   api_key="lm-studio", 
                   temperature=0)

#### Python Functions

In [7]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

#### Pydantic class

In [8]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

#### Typedict class

In [9]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [10]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='The result of 3 * 12 is 36.\n\nTo calculate this, we multiply the two numbers together:\n\n3 * 12 = (3 × 10) + (3 × 2)\n       = 30 + 6\n       = 36\n\nSo, 3 multiplied by 12 equals 36.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 17, 'total_tokens': 83, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-9d5f9405-6a05-40f3-8096-a0b79f66cad6-0', usage_metadata={'input_tokens': 17, 'output_tokens': 66, 'total_tokens': 83})

#### Tool calls

In [11]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[]

#### Parsing

In [12]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[]

The next few cells are from a follow up document called [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

I append the code into this document just to keep the notes in one notebook.

In [13]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [14]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[]


In [15]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='The answer to 3 * 12 is 36. To calculate this, multiply the two numbers: 3 × 12 = 36.\n\nThe answer to 11 + 49 is 60. To calculate this, add the two numbers: 11 + 49 = 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 27, 'total_tokens': 87, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b10cb922-a87a-448a-87ac-25284b7cfedb-0', usage_metadata={'input_tokens': 27, 'output_tokens': 60, 'total_tokens': 87})]

In [16]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 0, 'prompt_tokens': 87, 'total_tokens': 87, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc55232d-8e4d-4f61-b33a-c71426c59552-0', usage_metadata={'input_tokens': 87, 'output_tokens': 0, 'total_tokens': 87})